# Aufgabe 14: Lösungsvorschlag

In [1]:
using LinearAlgebra
using Random
using Permutations

Random.seed!(0815)
m = 4000
A = randn(m, m)

det(A)

Inf

Da Überlauf und Rechnen mit `Inf` das Vorzeichen korrekt behandelt, ist sicherlich $s=1$. Wer es nicht glaubt, rechnet halt nochmal nach...

In [2]:
F = lu(A);                                        # die Grundlage der Determinanten-Berechnung
s = sign(Permutation(F.p))*prod(sign.(diag(F.U))) # Vorzeichen

1.0

In [3]:
λ = sum(log10.(abs.(diag(F.U)))) # log₁₀(|det(A)|)

6334.56580228379

Dieses Verfahren steckt auch hinter dem Befehl `logabsdet` (natürlicher Logarithmus), der als Argument die Faktorisierung von $A$ akzeptiert:

In [4]:
logabsdet(F)./(log(10), 1)  # Umrechnung auf log₁₀ : ergibt (λ, s)

(6334.56580228379, 1.0)

Für die Beurteilung der Genauigkeit (bei gegebenem $R$-Faktor...) ermitteln wir die Kondition der Summe für $\lambda$

In [5]:
κ = sum(abs.(log10.(abs.(diag(F.U)))))/abs(λ)  # Kondition der Summe für λ

1.0

Und hier die Mantisse (auf $9 = 1+8$ Stellen gerundet, wie im PDF des Lösungsvorschlags erläutert) und der Exponent von $\det(\hat A)=s\cdot 10^\lambda$:

In [6]:
ex = floor(λ)
round(10^(λ-ex), digits=8), ex

(3.67961418, 6334.0)

### Alternative Lösung: Skalierung

Über- und Unterlauf lassen sich auch durch _Skalierung_ vermeiden. Diese ist für Determinanten wegen des Skalierungsverhaltens $\det(A)= \alpha^{m}\det(\alpha^{-1} A)$ für große Dimensionen $m$ aber notorisch schwierig. Ohne Verwendung von `logabsdet` ist es daher äußerst schwer, ein brauchbares $\alpha$ zu finden. Aber in Kenntnis der Größenordnung $10^{6334}$ ist dies jetzt möglich:

In [7]:
logα = round(ex/m, digits=4)
(det(A/10^logα), m*logα)

(3.679614179785332, 6334.0)

Wir können so aber kaum an Genauigkeit gewinnen: Zwar entfällt die Subtraktion des ganzzahligen Anteils $6334$, im Gegenzug erhöht sich aber die Kondition der Summe für $\lambda$ um den Faktor $\approx 1000$ (dies ist bei näherem Nachdenken natürlich kein Zufall), was einer Einbuße von 3 Ziffern entspricht: 

In [8]:
F = lu(A/10^logα)
λ = sum(log10.(abs.(diag(F.U))))
κ = sum(abs.(log10.(abs.(diag(F.U)))))/abs(λ)

1178.3184615554235

Wir erhalten so eine Genauigkeit von $13-3 = 10$ Ziffern für $\det(\hat A)$:

In [9]:
(round(det(F), digits=9), m*logα)

(3.67961418, 6334.0)

## Weitergehende Diskussion der erzielten Genauigkeit

Wenn die Angabe der belastbaren Ziffern eines numerisch stabilen Resultats nicht ausreicht und eine  Stabilitäts- und Konditionsanalyse mit realistischen Abschätzungen nicht zur Verfügung steht, dann muss man die Sensitivität von Problem & Algorithmus durch den Vergleich mit einer _zweiten_ Maschinengenauigkeit „auf den Prüfstand stellen“. Für eine solche Kontrollrechnung gibt es zwei Möglichkeiten: weniger genau oder genauer.

#### Variante 1 der Kontrollrechnung: einfach genau

In [10]:
AA = Float32.(A)

λλ, s = logabsdet(AA)./(log(10.f0), 1) 

ee = floor(λλ)
10.f0^(λλ-ee), ee

(3.6353555f0, 6334.0f0)

Wir sehen eine Übereinstimmung von etwas weniger als 2 Ziffern mit dem doppelt-genauen Resultat. Bei einer Mantissenlänge von etwas weniger als 8 Ziffern haben wir hier also in etwa 6 Ziffern Genauigkeit durch die Kondition des Problems _und_ das Rundungsfehlerverhalten des Algorithmus verloren. Setzen wir den gleichen Verlust auch bei der doppelt-genauen Rechnung an, so gelangen wir zu mindestens 10 korrekten Ziffern 

$$\det(A) \Doteq 3.679614180 \cdot 10^{6334}.$$

Beachte, dass nun wirklich auch $\det(A)$ und nicht wie oben $\det(\hat A)$ (das ja von der gewählten Maschinengenauigkeit abhängt) gemeint ist.

#### Variante 2 der Kontrollrechnung: vierfach genau (sehr sehr viel langsamer und völlig unnötig...)

In [11]:
using Quadmath       # implementiert IEEE754-2008 quadruple precision in Software (gnu libquadmath)
                     # (entspricht einer Genauigkeit von ca. 34 Dezimalstellen) 

AA = Float128.(A)

λλ, s = logabsdet(AA)./(log(Float128(10)), 1) 
ee = floor(λλ)
Float128(10)^(λλ-ee)

3.67961417978457225472287975862788323e+00

Wir beobachten hier eine Übereinstimmung von etwas mehr als 11 Ziffern mit der doppelt-genauen Rechnung, d.h. diese hatte tatsächlich nur etwas mehr als 5 Ziffern verloren. Beachte die Konsistenz dieses Ergebnisses mit der Variante 1 unserer Kontrollrechnung: vierfache Genauigkeit ist im vorliegenden Fall allein zur Beurteilung der Genauigkeit der doppelt-genauen Rechnung völliger „Overkill“. 

Variante 2 ist nur dann empfehlenswert, wenn sich mit Variante 1 kein Ergebnis erzielen lässt (d.h. im Fall, dass das betrachtete Problem zu schlecht konditioniert und/oder der verwendete Algorithmus zu instabil ist).